In [1]:
# install packages
! pip install pandas==1.1.5
! pip install xgboost==1.4.2
! pip install numpy==1.19.5

In [47]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import recall_score, precision_score

import pickle
import os

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = os.getcwd()

# Check if running in Colab
try:
  from google.colab import drive
  IN_COLAB=True
  print("Running in Colab")
  # Mount Google Drive 
  drive.mount('/content/drive')
  # Change directory
  path = "/content/drive/My \Drive/BitterMatch"
  %cd /content/drive/My \Drive/BitterMatch
except:
  IN_COLAB=False
  print("Running locally")

Running in Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/BitterMatch


In [4]:
from similarity import *
from preprocessing import *

## Load Original Data

Load and prepare $A, X^{\text{Rec}}, X^{\text{Lig}}$

In [5]:
# load known association for other ligands
A = load_A('Data/bitterdb_associations.csv')

# load ligand features
X_Lig = load_X_Lig('Data/ligand_features.csv')

# load receptor features
X_Rec = load_X_Rec('Data/receptor_features.csv')

In [6]:
# remove all_zero ligands
human = A.iloc[:,A.columns<2000]
mouse = A.iloc[:,A.columns>2000]

is_non_zero = (np.sum(human,axis=1)>0)
A = A[is_non_zero]

Load and prepare $S^{\text{Rec}}_1, \dots, S^{\text{Rec}}_4$

In [7]:
# receptor pre-computed similarities
Rec_sim = read_receptor_similarity('Data/Columns_similarity_matrix.csv')
Rec_ident = read_receptor_similarity('Data/Columns_identity_matrix.csv')
Rec_sites_sim = read_receptor_similarity('Data/Col_Similarity_Bindingsite.csv')
Rec_sites_ident = read_receptor_similarity('Data/Col_Identity_Bindingsites.csv')

In [8]:
# remove orphan receptors
orphan_receptors = A.columns[np.sum(A, axis=0)==0]

A = A[A.columns[~np.isin(A.columns, orphan_receptors)]]
X_Rec = X_Rec[~np.isin(X_Rec.column_label, orphan_receptors)]

In [9]:
# create data frame of all pairs of ligand-receptor
pairs = pd.melt(A.assign(ligand=A.index), id_vars='ligand', var_name='receptor', value_name='association')
pairs['test'] = np.repeat(False, len(pairs))

## Load Model

In [10]:
# load model
xgb_clf = xgb.XGBClassifier(objective = 'binary:logistic',  booster = 'gbtree')
xgb_model = pickle.load(open("Trained_Models/trained_model_for_new_ligands.pkl", "rb"))

## Load additional Data

In [12]:
# chemica properties of new ligands
new_X_Lig = load_X_Lig('Data/Validation/Ligand_features.csv')

# new associations (if no associations are known an empty matrix should be created instead)
new_A = load_A('Data/Validation/associations.csv')
# remove orphan receptors
new_A = new_A[new_A.columns[~np.isin(new_A.columns, orphan_receptors)]]

In [13]:
# load ligand similarities (including previously known)
Lig_linear_sim = pd.read_csv('Data/Validation/Ligand_linear_limilarity_combined.csv', sep=',') 
Lig_mol2d_sim = pd.read_csv('Data/Validation/Ligand_Mol2D_limilarity_combined.csv', sep=',')

In [14]:
# name lists
all_names_str = Lig_linear_sim[Lig_linear_sim.columns[0]].values
new_names_str = new_X_Lig.cid.values
old_names_str = [a for a in all_names_str if a not in new_names_str]

In [15]:
n_new = len(new_X_Lig)

In [16]:
# numerical ids for new ligands
new_ligand_names = new_A.index.values
new_ligand_ids = [999000 + i for i in range(len(new_ligand_names))]

# replace names with numerical ids in association matrix
new_A.index = new_ligand_ids
new_A.columns = [int(v) for v in new_A.columns]

# create names and ids dictionaries
name_dict = {}
for i, name in enumerate(new_ligand_names):
    name_dict[name] = new_ligand_ids[i]
rev_name_dict = {v: k for k, v in name_dict.items()}

# replace names with numerical ids in ligand features and in structural classifications
new_X_Lig = new_X_Lig.replace({"cid": name_dict})
# new_families_df.index = [name_dict[v] for v in new_families_df.index]

In [17]:
# integer names (to be used in similarity matrices)
all_names_int = []
for name in all_names_str:
    if name in new_names_str:
        v = int(name_dict[name])
        all_names_int.append(v)
    else:
        v = int(name)
        all_names_int.append(v)

In [18]:
# renaming function
def rename_ligand_sim(df, n_new, all_names):
    df.iloc[:,0] = all_names
    df.columns = ['Unnamed'] + all_names
    df.index = np.array(all_names).astype('int64')
    df.drop('Unnamed', inplace=True, axis=1)
    df.columns = np.array(all_names).astype('int64')

In [19]:
# rename ligands in pre-computed similarity matrices
rename_ligand_sim(Lig_linear_sim, n_new, all_names_int)
rename_ligand_sim(Lig_mol2d_sim, n_new, all_names_int)

In [20]:
# preprocessing of similarity matrices (for new and old ligands)
Lig_linear_sim = read_ligand_similarity(Lig_linear_sim, from_file=False)
Lig_mol2d_sim = read_ligand_similarity(Lig_mol2d_sim, from_file=False)

In [21]:
# merge ligand features 
X_Lig = pd.concat([X_Lig, new_X_Lig])

## Prepare data for prediction

In [22]:
new_pairs = pd.melt(new_A.assign(ligand=new_A.index), id_vars='ligand', var_name='receptor', value_name='association')

In [23]:
# limit data to known associations 
new_pairs = new_pairs[pd.Series.notna(new_pairs.association)]

In [24]:
new_A = pd.DataFrame(np.nan, index=new_A.index, columns=new_A.columns)

In [25]:
pairs = pd.concat([pairs, new_pairs])
A = pd.concat([A, new_A])

In [26]:
# restrict ligand similarities only to those in A
Lig_linear_sim = Lig_linear_sim.iloc[np.isin(Lig_linear_sim.index, A.index), np.isin(Lig_linear_sim.columns, A.index)]
Lig_mol2d_sim = Lig_mol2d_sim.iloc[np.isin(Lig_mol2d_sim.index, A.index), np.isin(Lig_mol2d_sim.columns, A.index)]

In [27]:
pairs = pairs.astype({'ligand':'int64','receptor':'int64','association':'float64','test':'bool'})

In [28]:
pairs.test[pairs.receptor > 2000] = False

In [29]:
# cross join ligand and receptor data
features_df = pd.merge(X_Lig.rename(columns=lambda l: 'Lig_%s' % l).assign(key_=1),
                       X_Rec.rename(columns=lambda r: 'Rec_%s' % r).assign(key_=1),
                       on='key_').drop('key_', 1)
features_df = features_df.rename(columns={'Lig_cid': 'ligand', 'Rec_column_label': 'receptor'})

# add a feature indicating whether the receptor is human
features_df['is_human_receptor'] = features_df.receptor<2000


# merge full association (ground truth) and train/test split
features_df = features_df.merge(pairs, how='outer', on=['ligand', 'receptor'])

In [30]:
# keep only test pairs
features_df = features_df[features_df.test == True]

In [31]:
sim_metrics_dict = {'Lig_linear_sim':       (Lig_linear_sim,   0),
                    'Lig_mol2d_sim':        (Lig_mol2d_sim,    0)}

for prefix, (sim_df, axis) in sim_metrics_dict.items():
    sim_metrics_df = sim_metrics(sim_df, A, axis).rename(columns=lambda col: '%s_%s' % (prefix, col))
    features_df = features_df.merge(sim_metrics_df, how='left', on=['ligand', 'receptor'])

In [32]:
results_df = features_df[(features_df.test == True) & (features_df.association.isna() == False)]
X_test = features_df[(features_df.test == True) & (features_df.association.isna() == False)].drop(['ligand', 'receptor', 'association', 'test'], axis=1)
Y_test = features_df[(features_df.test == True) & (features_df.association.isna() == False)].association.values

In [33]:
X_test.is_human_receptor = X_test.is_human_receptor.astype('bool')

## Prediction and stats

In [43]:
Y_pred = xgb_model.predict_proba(X_test)
Y_pred = Y_pred[:,1]
results_df['pred_score'] = Y_pred

t = 0.5248
results_df['pred_score'] = Y_pred
results_df['pred'] = 1*(Y_pred >= t)

In [44]:
slim_results_df = results_df[['ligand', 'receptor', 'association', 'pred', 'pred_score']]
slim_results_df['ligand'] = slim_results_df['ligand'].replace(rev_name_dict)

In [45]:
slim_results_df.head()

,ligand,receptor,association,pred,pred_score
0,2-Acetylbenzofuran,1,0.0,0,0.250107
1,2-Acetylbenzofuran,3,0.0,0,0.210610
2,2-Acetylbenzofuran,4,0.0,0,0.247901
3,2-Acetylbenzofuran,5,0.0,0,0.215405
4,2-Acetylbenzofuran,7,0.0,0,0.229155


In [48]:
y = slim_results_df.association
y_hat = slim_results_df.pred

recall = recall_score(y, y_hat)
precision = precision_score(y, y_hat)
print("recall {}, precision {}".format(recall, precision))

recall 0.42105263157894735, precision 0.8
